In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyarrow.parquet as pq
import sys
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import math
from IPython.display import clear_output
import logging
from pathlib import Path
from functools import partial

import matplotlib.pyplot as plt
sys.path.append('../../../')

from configs.data_configs.rosbank import data_configs
from configs.model_configs.gen.rosbank import model_configs
from src.models.mTAND.model import MegaNet, MegaNetCE, MegaNetSupervised
from src.data_load.dataloader import create_data_loaders, create_test_loader
from src.trainers.trainer_mTAND import MtandTrainer

from src.create_embeddings import create_embeddings

from src.data_load import split_strategy
from src.data_load.data_utils import prepare_data
from src.data_load.splitting_dataset import (
    ConvertingTrxDataset,
    TargetDataset,
    DropoutTrxDataset,
    SplittingDataset,
    TargetEnumeratorDataset,
)
from src.data_load.dataloader import collate_splitted_rows, padded_collate, PaddedBatch
from torch.utils.data import DataLoader

from sklearn.linear_model import LogisticRegression

from src.models.preprocessors import FeatureProcessor

from torch.autograd import Variable

from src.models.gen_models import SeqGen

from sklearn.metrics import roc_auc_score

/home/event_seq/experiments/rosbank/notebooks/../../../src/trainers/base_trainer.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
x = torch.rand(3, 5, 7) 

In [7]:
np.zeros([5,2])

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [ ]:
def random_generator(batch_size, z_dim, T_mb, max_seq_len):
  """Random vector generation.
  
  Args:
    - batch_size: size of the random vector
    - z_dim: dimension of random vector
    - T_mb: time information for the random vector
    - max_seq_len: maximum sequence length
    
  Returns:
    - Z_mb: generated random vector
  """
  Z_mb = list()
  for i in range(batch_size):
    temp = np.zeros([max_seq_len, z_dim])
    temp_Z = np.random.uniform(0., 1, [T_mb[i], z_dim])
    temp[:T_mb[i],:] = temp_Z
    Z_mb.append(temp)
  return torch.tensor(np.stack(Z_mb))

class D(nn.Module):

    def __init__(self):
        super().__init___()
        self.rnn = nn.GRU(input_size = 9, hidden_size=9, batch_first=True)

        self.proj = nn.Sequential(
            nn.Linear(9, 12),
            nn.ReLU(),
            nn.Linear(12, 1)
        )

    def forward(self, x):


class TG(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoder = nn.GRU(input_size=7, hidden_size=9, batch_first=True)
        self.decoder = nn.Sequential(
            nn.Linear(9, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(9, 7)
        )
        self.generator = nn.GRU(input_size=7, hidden_size=9, batch_first=True)
    def forward(self, x):
        latens, hn = self.encoder(x)
        decoded = self.decoder(latens)

        bs, l, d = x.size()
        Z = random_generator(bs, d, [l]*bs, l)
        gen_latens = self.generator(Z)


In [26]:
z = random_generator(3, 7, [4, 4, 5], 5)

In [27]:
z

tensor([[[0.4727, 0.5375, 0.2085, 0.6701, 0.7857, 0.2511, 0.1718],
         [0.8589, 0.6937, 0.2587, 0.0881, 0.2629, 0.5737, 0.6829],
         [0.3491, 0.2518, 0.7863, 0.1141, 0.9193, 0.1637, 0.7340],
         [0.3878, 0.6488, 0.8230, 0.9885, 0.6167, 0.3225, 0.7368],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.9500, 0.6378, 0.7989, 0.4544, 0.9919, 0.5176, 0.1610],
         [0.7488, 0.2407, 0.7099, 0.4868, 0.9233, 0.1697, 0.4151],
         [0.2066, 0.9919, 0.8938, 0.0862, 0.2189, 0.6643, 0.1941],
         [0.2517, 0.4732, 0.5657, 0.8848, 0.4648, 0.2628, 0.5808],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.5084, 0.1570, 0.9775, 0.2200, 0.4831, 0.9544, 0.8391],
         [0.3010, 0.2247, 0.8673, 0.8620, 0.8760, 0.5144, 0.0328],
         [0.2851, 0.3758, 0.5251, 0.4691, 0.4919, 0.1496, 0.7183],
         [0.1576, 0.8898, 0.9043, 0.8255, 0.0421, 0.8710, 0.9014],
         [0.9034, 0.3027, 0.1185, 0.2005, 0.0529, 0.8897, 